# Credit Card Anomaly Detection via Feature Creation


---
## Exective Summary


This project analyzes 442,458 purchase card transactions from state and higher-education institutions, spanning July 2013 – June 2014, to detect anomalies and strengthen internal controls.

It is crucial because government purchase cards accounted for over **\$17 billion** in micropurchases in 2005–06—representing **57 percent** of total spend([gao.gov][1])—yet **41 percent** of transactions lacked proper authorization or documentation, exposing agencies to fraud and waste([gao.gov][1], [gao.gov][2]). Early detection of unusual patterns can save millions and ensure compliance with federal guidelines.

We explored multiple methods:

* **Descriptive analytics** (monthly spend, vendor and MCC breakdowns)
* **Anomaly-focused features** (transaction velocity, refund ratios, ratio-to-vendor/MCC median, and many more)
* **Percentile‐based binning and thresholding** for outlier detection

Key results included identifying:

* **0.8 percent** of vendors and **27 percent** of cardholders with anomalous daily transaction spikes
* Rare long posting lags (> 129 days) tied mainly to refunds or corrections
* Vendor‐MCC ratio outliers in the top 0.1 percent of transactions

**Recommendations & Impact**

* **Design and implement preventive controls** to reduce the risk of fraud and waste found in the analysis
* **Implement automated monitoring** of velocity and refund-ratio flags to catch misuse in real time
* **Enforce stricter documentation** and approval workflows, especially for one-off large transactions
* **Regularly review posting lags** to distinguish genuine delays from back-dated entries


By embedding these controls, agencies can reduce erroneous or fraudulent purchases, improve audit readiness, and safeguard public funds.

---



## Load Packages

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"
import numpy as np


## Set Configs

In [3]:
# Suppress scientific notation globally for floats
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None

data_dir = "/Users/mrla/Documents/Projects/data/credit_card/"

## Load Data

This dataset contains information on purchases made through the purchase card programs administered by the state and higher ed institutions. The purchase card information will be updated monthly after the end of the month. For example, July information will be added in August.

This data can be obtained [here](https://data.ok.gov/dataset/purchase-card-pcard-fiscal-year-2014)

In [4]:
df = pd.read_csv(data_dir + "purchase_credit_card.csv")

In [5]:
print(f"Shape of data: {df.shape[0]:,} rows, {df.shape[1]:,} columns")
print(f"Columns in data: {df.columns.tolist()}")
print(f"First 5 rows of data:\n{df.head()}")


Shape of data: 442,458 rows, 11 columns
Columns in data: ['Year-Month', 'Agency Number', 'Agency Name', 'Cardholder Last Name', 'Cardholder First Initial', 'Description', 'Amount', 'Vendor', 'Transaction Date', 'Posted Date', 'Merchant Category Code (MCC)']
First 5 rows of data:
   Year-Month  Agency Number                Agency Name Cardholder Last Name  \
0      201307           1000  OKLAHOMA STATE UNIVERSITY                Mason   
1      201307           1000  OKLAHOMA STATE UNIVERSITY                Mason   
2      201307           1000  OKLAHOMA STATE UNIVERSITY               Massey   
3      201307           1000  OKLAHOMA STATE UNIVERSITY               Massey   
4      201307           1000  OKLAHOMA STATE UNIVERSITY        Mauro-Herrera   

  Cardholder First Initial                   Description  Amount  \
0                        C              GENERAL PURCHASE  890.00   
1                        C                  ROOM CHARGES  368.96   
2                        J         

## Set Date Data Types

We will convert all date columns to the `datetime` data type. This will allow us to perform time-based operations and analyses on the data.



In [6]:
# Convert 'Year-Month' to datetime, setting day to 1
df['Year-Month'] = pd.to_datetime(df['Year-Month'].astype(str), format='%Y%m').dt.to_period('M').dt.to_timestamp()

# Convert 'Transaction Date' and 'Posted Date' to datetime
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], format='%m/%d/%Y %I:%M:%S %p')
df['Posted Date']      = pd.to_datetime(df['Posted Date'],      format='%m/%d/%Y %I:%M:%S %p')


## Exploratory Data Analysis

In this section, we will explore high level trends in the data. We will look at multiple key informations like the number of transactions, total amount spent, and average transaction amount by month, etc.

### Total Amount per Year Month

In [7]:
# Function to compute boxplot statistics
def boxplot_stats(group):
    q1 = group.quantile(0.25)
    q3 = group.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    outliers = ((group < lower) | (group > upper)).sum()
    return pd.Series({
        '01.count': group.count(),
        '02.min': group.min(),
        '03.Q1': q1,
        '04.median': group.median(),
        '05.Q3': q3,
        '06.max': group.max(),
        '07.IQR': iqr,
        '08.lower_bound': lower,
        '09.upper_bound': upper,
        '10.num_outliers': outliers
    })

# Step 2: Apply and reformat
summary_df = df.groupby('Year-Month')['Amount'].apply(boxplot_stats).reset_index()
summary_pivot = summary_df.pivot(index='level_1', columns='Year-Month', values='Amount')

# Optional: format column headers to YYYY-MM
summary_pivot.columns = [col.strftime('%Y-%m') for col in summary_pivot.columns]

# Optional: round for readability
summary_pivot = summary_pivot.round(2)

# Final result
display(summary_pivot)

,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06
level_1,,,,,,,,,,,,
01.count,"37,635.00","39,314.00","38,762.00","40,266.00","34,275.00","26,969.00","37,230.00","35,831.00","38,188.00","39,249.00","36,784.00","37,955.00"
02.min,"-38,506.87","-18,899.00","-33,075.32","-7,188.61","-7,860.22","-3,718.00","-27,864.00","-34,108.00","-42,863.04","-10,140.00","-4,889.36","-21,000.00"
03.Q1,30.32,32.64,30.85,30.00,31.08,29.99,30.00,30.74,31.79,31.42,31.25,32.36
04.median,105.00,107.02,102.35,100.00,100.00,103.00,101.98,105.12,110.00,107.95,106.98,108.80
05.Q3,350.00,346.31,342.99,325.00,316.00,338.04,337.49,339.00,361.83,360.50,353.84,359.82
06.max,"343,148.50","1,750,379.98","1,903,858.37","1,089,180.00","335,197.99","281,185.00","306,143.75","306,165.68","855,343.00","373,150.26","348,053.75","132,790.14"
07.IQR,319.68,313.67,312.14,295.00,284.92,308.05,307.49,308.26,330.04,329.08,322.59,327.45
08.lower_bound,-449.19,-437.86,-437.35,-412.50,-396.30,-432.08,-431.24,-431.64,-463.27,-462.20,-452.64,-458.81
09.upper_bound,829.51,816.81,811.19,767.50,743.38,800.12,798.73,801.38,856.89,854.12,837.73,850.99


In [8]:
# Aggregate monthly data
monthly_summary = df.groupby('Year-Month').agg(
    total_amount=('Amount', 'sum'),
    transaction_count=('Amount', 'count')
).reset_index()

# Create the dual-axis chart
fig = go.Figure()

# Line for total amount
fig.add_trace(go.Bar(
    x=monthly_summary['Year-Month'],
    y=monthly_summary['total_amount'],
    name='Total Amount ($)',
    yaxis='y1',
    marker_color='steelblue'
))

# Line for transaction count
fig.add_trace(go.Scatter(
    x=monthly_summary['Year-Month'],
    y=monthly_summary['transaction_count'],
    name='Transaction Count',
    yaxis='y2',
    mode='lines+markers',
    line=dict(color='darkorange')
))

# Layout with two y-axes
fig.update_layout(
    title='Monthly Total Amount and Transaction Count',
    xaxis=dict(title='Month'),
    yaxis=dict(title='Total Amount ($)', side='left'),
    yaxis2=dict(
        title='Transaction Count',
        overlaying='y',
        side='right',
        showgrid=False,
        range=[0, monthly_summary['transaction_count'].max() * 1.1]
    ),
    legend=dict(x=0.01, y=0.99),
    template='plotly_white'
)

fig.show()

* Nov and Dec are the months with the lowest transaction amounts and volumes.
* March, September, and Oct are the months with the highest transaction amounts and volumes.

### Amount per Agency

In [9]:
# Aggregate total amount per agency
agency_totals = df.groupby(['Agency Number', 'Agency Name'])['Amount'].sum().reset_index()

# Optional: sort by total amount
agency_totals = agency_totals.sort_values('Amount', ascending=True)

# Create readable label: "Agency Number – Agency Name"
agency_totals['Agency Label'] = agency_totals['Agency Number'].astype(str) + ' – ' + agency_totals['Agency Name']

fig = px.bar(
    agency_totals.tail(20),
    y='Agency Label',    # categorical axis
    x='Amount',          # numeric axis
    orientation='h',     # vertical bars
    title='Top 20 Total Purchase Amount per Agency',
    labels={'Amount': 'Total Amount (USD)', 'Agency Label': 'Agency'},
    height=600
)

# Rotate x-axis labels for readability if there are many agencies
fig.update_layout(
    xaxis_tickangle=-45,
    xaxis_tickfont=dict(size=10),
    margin=dict(l=40, r=40, t=60, b=60),
)

fig.show()

* Oklahoma State University has the highest total amount spent, followed by the University of Oklahoma and the Department of Health Sciences form University of Oklahoma.

### Amount per Description

In [10]:

desc_totals = (
    df.groupby('Description')['Amount']
      .sum()
      .reset_index()
      .sort_values('Amount', ascending=False)
)

# If there are too many descriptions, you can take the top N, e.g.:
# desc_totals = desc_totals.head(20)

# Plot horizontal bar chart: descriptions on y-axis, amount on x-axis
fig = px.bar(
    desc_totals.head(20),
    y='Description',
    x='Amount',
    orientation='h',
    title='Top 20 Total Purchase Amount per Description',
    labels={'Amount': 'Total Amount (USD)', 'Description': 'Transaction Description'},
    template='plotly_white',
    height=800
)

# Invert the y-axis so the largest bars appear at the top
fig.update_layout(
    yaxis=dict(autorange='reversed'),
    margin=dict(l=300, r=40, t=60, b=40)
)

fig.show()

The vast majority of the transactions made here are cartegorized as "General Purchase" followed far behind by "Travel" and "Room Charges".

### Amount per Vendor

In [11]:
vendor_totals = (
    df.groupby('Vendor')['Amount']
      .sum()
      .reset_index()
      .sort_values('Amount', ascending=False)
)

# Optionally, limit to top N vendors to keep the chart readable
# vendor_totals = vendor_totals.head(20)

# Plot horizontal bar chart: vendors on y-axis, total amount on x-axis
fig = px.bar(
    vendor_totals.head(40),
    y='Vendor',
    x='Amount',
    orientation='h',
    title='Top 20 Total Purchase Amount per Vendor',
    labels={'Amount': 'Total Amount (USD)', 'Vendor': 'Vendor Name'},
    template='plotly_white',
    height=800
)

# Invert the y-axis so the highest spenders are at the top
fig.update_layout(
    yaxis=dict(autorange='reversed'),
    margin=dict(l=300, r=40, t=60, b=40)
)

fig.show()

* Transaction value by Vendor are more stable
* Graingner is the vendor with the highest total amount spent on

### MCCs per Vendors

In [12]:
mcc_vendors = df.groupby(['Vendor'])['Merchant Category Code (MCC)'].nunique().reset_index(name='n_mccs').sort_values('n_mccs', ascending=False)

print(mcc_vendors.head(10))
print("\n")
print(f"Average MCCs per vendor are {mcc_vendors['n_mccs'].mean():.2f}")

                        Vendor  n_mccs
24574               CLAIM ADJ/       8
25763  CREDIT PURCHASE BALANCE       5
62769      PAYPAL  ASSOCIATION       4
56369  NATIONAL ASSOCIATION OF       4
43661      HOLIDAY INN EXPRESS       3
40743   FIVE STAR OFFICE SUPPL       3
77315         TALX CORPORATION       3
46202                   IVECCS       3
63939      PAYPAL  REGISTRYINC       3
19582  AMERICAS BEST VALUE INN       3


Average MCCs per vendor are 1.01


The majority of vendors only have one MCC code, but there are some vendors with multiple MCC codes

## Anomaly Detection Using Feature Engineering

### 1. Ratio to Avg Transaction by Merchant x Merchant Category Code

We will begin by checking the ratio to avg transactions by merchant and merchant category code (MCC). This will help us identify transactions that are significantly higher or lower than the average for a given merchant and MCC combination.

In [14]:
# Compute average Amount for each (Vendor, MCC) combination
vendor_mcc_avg = (
    df
    .groupby(['Vendor', 'Merchant Category Code (MCC)'])['Amount']
    .mean()
    .reset_index(name='vendor_mcc_avg')
    .sort_values(by=['Vendor', 'Merchant Category Code (MCC)'], ascending=False)
)

print(vendor_mcc_avg.head(10))

print("\n---------------------------\n")

# Merge the average back onto the original DataFrame
df1 = df.merge(
    vendor_mcc_avg,
    on=['Vendor', 'Merchant Category Code (MCC)'],
    how='left'
)

# Compute the ratio of each transaction to its group average
df1['ratio_to_vendor_mcc_avg'] = np.where(df1['vendor_mcc_avg'] == 0, 0, df1['Amount'] / df1['vendor_mcc_avg'])

# Inspect the results
print(
    df1[
        ['Vendor', 'Merchant Category Code (MCC)', 'Amount', 'vendor_mcc_avg', 'ratio_to_vendor_mcc_avg']
    ].head(10)
)

print("\n---------------------------\n")

print(df1['ratio_to_vendor_mcc_avg'].describe())

                         Vendor  \
87411             www.wiley.com   
87410     www.wff2014korea.org/   
87409          www.testking.com   
87408      www.sublimevideo.net   
87407           www.molport.com   
87406      www.eurofedlipid.org   
87405            www.autodoc.ch   
87404  wmill.com ResponseOmatic   
87403        value plastics inc   
87402       templatemonster.com   

                            Merchant Category Code (MCC)  vendor_mcc_avg  
87411         BUSINESS SERVICES NOT ELSEWHERE CLASSIFIED          596.78  
87410  DIRCT MARKETING/DIRCT MARKETERS--NOT ELSEWHERE...          250.00  
87409         BUSINESS SERVICES NOT ELSEWHERE CLASSIFIED           99.99  
87408                           COMPUTER SOFTWARE STORES           18.19  
87407  CHEMICALS AND ALLIED PRODUCTS NOT ELSEWHERE CL...          205.00  
87406                  MISCELLANEOUS GENERAL MERCHANDISE        1,286.43  
87405                           COMPUTER SOFTWARE STORES          174.89  
87404     PROF

Interestingly there are some transactions that have a very high and very low ratio of transaction amount to the average transaction amount for that merchant and MCC code. This could indicate potential anomalies or unusual spending patterns.

In [15]:
# Define your percentiles
percentiles = [0, 0.0001, 0.0005, 0.01, 0.05, 0.2, 0.5, 0.8, 0.95, 0.99, 0.9999, 0.99995, 1.0]

# Generate human-readable bin labels
bin_labels = [
    f"{percentiles[i]*100:.4f}–{percentiles[i+1]*100:.4f}%"
    for i in range(len(percentiles) - 1)
]

# Create the binned variable on df1
df1['ratio_bin'] = pd.qcut(
    df1['ratio_to_vendor_mcc_avg'],
    q=percentiles,
    labels=bin_labels
)

# Count number of records in each bin
bin_counts = (
    df1['ratio_bin']
    .value_counts()
    .sort_index()
    .reset_index(name='count')
    .rename(columns={'index': 'ratio_bin'})
)

# Plot with Plotly
fig = px.bar(
    bin_counts,
    x='ratio_bin',
    y='count',
    title='Transaction Count by Ratio-to-Average Percentile Bins',
    labels={'ratio_bin': 'Ratio Percentile Bin', 'count': 'Record Count'},
    template='plotly_white'
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()


print("\n---------------------------\n")
print("The ratio bins and their statistics in the ratio to vendor mcc average:")
display(df1.groupby('ratio_bin')['ratio_to_vendor_mcc_avg'].agg(['count', 'min', 'max']).reset_index())

print("\n---------------------------\n")
print("The extremely low ratios:")
display(df1[df1['ratio_to_vendor_mcc_avg'] < 0].sort_values('ratio_to_vendor_mcc_avg', ascending=True).head())

print("\n---------------------------\n")
print("The extremely high ratios:")
display(df1[df1['ratio_to_vendor_mcc_avg'] > 100].sort_values('ratio_to_vendor_mcc_avg', ascending=False).head())


---------------------------

The ratio bins and their statistics in the ratio to vendor mcc average:


/var/folders/sb/pvs4j23j1z74y7s6nw77zrwr0000gn/T/ipykernel_3936/1958716704.py:42: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,ratio_bin,count,min,max
0,0.0000–0.0100%,45,"-18,045,360,756,920,156.00",-30.05
1,0.0100–0.0500%,177,-29.19,-7.67
2,0.0500–1.0000%,4203,-7.63,-0.47
3,1.0000–5.0000%,17698,-0.47,0.03
4,5.0000–20.0000%,66369,0.03,0.21
5,20.0000–50.0000%,132738,0.21,0.79
6,50.0000–80.0000%,132736,0.79,1.20
7,80.0000–95.0000%,66369,1.20,2.96
8,95.0000–99.0000%,17698,2.96,6.98
9,99.0000–99.9900%,4380,6.98,59.17



---------------------------

The extremely low ratios:


,Year-Month,Agency Number,Agency Name,Cardholder Last Name,Cardholder First Initial,Description,Amount,Vendor,Transaction Date,Posted Date,Merchant Category Code (MCC),vendor_mcc_avg,ratio_to_vendor_mcc_avg,ratio_bin
311966,2014-03-01,53000,OKLA. PANHANDLE STATE UNIV.,Mauldin,D,GENERAL PURCHASE,-256.44,TRAVRES HotelPlanner,2014-03-24,2014-03-25,TRAVEL AGENCIES,0.00,"-18,045,360,756,920,156.00",0.0000–0.0100%
311967,2014-03-01,53000,OKLA. PANHANDLE STATE UNIV.,Mauldin,D,GENERAL PURCHASE,-256.44,TRAVRES HotelPlanner,2014-03-24,2014-03-25,TRAVEL AGENCIES,0.00,"-18,045,360,756,920,156.00",0.0000–0.0100%
311965,2014-03-01,53000,OKLA. PANHANDLE STATE UNIV.,Mauldin,D,GENERAL PURCHASE,-256.44,TRAVRES HotelPlanner,2014-03-24,2014-03-25,TRAVEL AGENCIES,0.00,"-18,045,360,756,920,156.00",0.0000–0.0100%
422223,2014-06-01,76000,UNIVERSITY OF OKLAHOMA,MCCASKILL,J,GENERAL PURCHASE,-6.63,GODADDY.COM CANADA,2014-06-19,2014-06-20,COMPUTER NETWORK/INFORMATION SERVICES,0.01,-663.00,0.0000–0.0100%
385422,2014-06-01,1000,OKLAHOMA STATE UNIVERSITY,Reece,S,GENERAL PURCHASE,"-1,876.34",BLUEBYTE TRADING LLC,2014-06-01,2014-06-03,MISCELLANEOUS AND SPECIALTY RETAIL STORES,2.84,-660.68,0.0000–0.0100%



---------------------------

The extremely high ratios:


,Year-Month,Agency Number,Agency Name,Cardholder Last Name,Cardholder First Initial,Description,Amount,Vendor,Transaction Date,Posted Date,Merchant Category Code (MCC),vendor_mcc_avg,ratio_to_vendor_mcc_avg,ratio_bin
311657,2014-02-01,53000,OKLA. PANHANDLE STATE UNIV.,Mauldin,D,GENERAL PURCHASE,769.32,TRAVRES HotelPlanner,2014-02-11,2014-02-12,TRAVEL AGENCIES,0.00,"54,136,082,270,760,472.00",99.9950–100.0000%
422221,2014-06-01,76000,UNIVERSITY OF OKLAHOMA,MCCASKILL,J,GENERAL PURCHASE,6.65,GODADDY.COM CANADA,2014-06-19,2014-06-20,COMPUTER NETWORK/INFORMATION SERVICES,0.01,665.00,99.9950–100.0000%
384436,2014-06-01,1000,OKLAHOMA STATE UNIVERSITY,Reece,S,GENERAL PURCHASE,"1,882.02",BLUEBYTE TRADING LLC,2014-05-31,2014-06-02,MISCELLANEOUS AND SPECIALTY RETAIL STORES,2.84,662.68,99.9950–100.0000%
3659,2013-07-01,1000,OKLAHOMA STATE UNIVERSITY,Brown,P,SHELVINGINDUSTRIAL36X1 NMB,-299.98,OFFICE DEPOT #47,2013-07-03,2013-07-05,"STATIONERY,OFFICE AND SCHOOL SUPPLY STORES",-0.69,436.16,99.9950–100.0000%
80683,2013-08-01,13100,DEPARTMENT OF CORRECTIONS,Bowlin,C,JDCC CHILLER PROJECT- (2) EA,"96,190.38",WW GRAINGER,2013-08-28,2013-08-30,INDUSTRIAL SUPPLIES NOT ELSEWHERE CLASSIFIED,421.45,228.24,99.9950–100.0000%


From the above sample we can see how many transactions are in the low and high ratio categories. 
Turns out the low and high ratio transactions are because the average vendor MCC amount is 0. This is odd because we have specifically set if the vendor mcc is 0, then the ratio is also 0. This issue is likely due to rounding errors (Python is showing us these numbers are 0 but are actually very close to 0).

In [16]:
df1[df1['vendor_mcc_avg'] == 0].head(10)

,Year-Month,Agency Number,Agency Name,Cardholder Last Name,Cardholder First Initial,Description,Amount,Vendor,Transaction Date,Posted Date,Merchant Category Code (MCC),vendor_mcc_avg,ratio_to_vendor_mcc_avg,ratio_bin
114,2013-07-01,1000,OKLAHOMA STATE UNIVERSITY,Henderson,K,GENERAL PURCHASE,-340.00,INTERWRX (INTERWRX.COM),2013-07-26,2013-07-29,TELECOMMUNICATION EQUIPMENT AND TELEPHONE SALES,0.00,0.00,1.0000–5.0000%
1779,2013-07-01,1000,OKLAHOMA STATE UNIVERSITY,Robedeaux,M,GENERAL PURCHASE,-537.00,MEADOWLAKE RANCH,2013-07-08,2013-07-09,RECREATION SERVICES--NOT ELSEWHERE CLASSIFIED,0.00,0.00,1.0000–5.0000%
1780,2013-07-01,1000,OKLAHOMA STATE UNIVERSITY,Robedeaux,M,GENERAL PURCHASE,537.00,MEADOWLAKE RANCH,2013-07-08,2013-07-09,RECREATION SERVICES--NOT ELSEWHERE CLASSIFIED,0.00,0.00,1.0000–5.0000%
7864,2013-07-01,1000,OKLAHOMA STATE UNIVERSITY,Henderson,K,GENERAL PURCHASE,340.00,INTERWRX (INTERWRX.COM),2013-07-22,2013-07-23,TELECOMMUNICATION EQUIPMENT AND TELEPHONE SALES,0.00,0.00,1.0000–5.0000%
10063,2013-08-01,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,AIR TRAVEL,438.60,DELTA AIR 0067295028873,2013-08-23,2013-08-26,DELTA,0.00,0.00,1.0000–5.0000%
10065,2013-08-01,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,AIR TRAVEL,20.00,AGENT FEE 8900593456183,2013-08-23,2013-08-26,AIRLINES AND AIR CARRIERS,0.00,0.00,1.0000–5.0000%
10066,2013-08-01,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,AIR TRAVEL,438.60,DELTA AIR 0067295028872,2013-08-23,2013-08-26,DELTA,0.00,0.00,1.0000–5.0000%
10101,2013-08-01,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,AIR TRAVEL,20.00,AGENT FEE 8900593456182,2013-08-23,2013-08-26,AIRLINES AND AIR CARRIERS,0.00,0.00,1.0000–5.0000%
10626,2013-08-01,1000,OKLAHOMA STATE UNIVERSITY,Hallgren,S,GENERAL PURCHASE,15.00,TACKLE SHACK,2013-08-28,2013-08-30,SPORTING GOODS STORES,0.00,0.00,1.0000–5.0000%
10714,2013-08-01,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,AIR TRAVEL,-438.60,DELTA AIR 0067295028873,2013-08-23,2013-08-28,DELTA,0.00,0.00,1.0000–5.0000%


Let's see what happens if we use median instead of average.

### 2. Ratio to Median Transaction by Merchant x Merchant Category Code

Same as the previous feature, but using median instead of average. This will help us identify transactions that are significantly higher or lower than the median for a given merchant and MCC combination.

In [17]:


# 1. Compute median Amount for each (Vendor, MCC) combination
vendor_mcc_median = (
    df
    .groupby(['Vendor', 'Merchant Category Code (MCC)'])['Amount']
    .median()
    .reset_index(name='vendor_mcc_median')
    .sort_values(by=['Vendor', 'Merchant Category Code (MCC)'], ascending=False)
)

print(vendor_mcc_median.head(10))
print("\n---------------------------\n")
print(f"Vendor MCC with Median 0: {vendor_mcc_median[(vendor_mcc_median['vendor_mcc_median'] == 0) & 
                                                     (vendor_mcc_median['vendor_mcc_median'] != 0.01)].shape[0]:,} rows")

print("\n---------------------------\n")

# 2. Merge the median back onto the original DataFrame
df2 = df.merge(
    vendor_mcc_median,
    on=['Vendor', 'Merchant Category Code (MCC)'],
    how='left'
)

# 3. Compute the ratio of each transaction to its group median
df2['ratio_to_vendor_mcc_median'] = np.where(
    df2['vendor_mcc_median'] == 0,
    0,
    df2['Amount'] / df2['vendor_mcc_median']
)

# 4. Inspect the results
print(df2[['Vendor', 'Merchant Category Code (MCC)', 'Amount', 
           'vendor_mcc_median', 'ratio_to_vendor_mcc_median']].head(10))
print("\n---------------------------\n")
print(df2['ratio_to_vendor_mcc_median'].describe())
print("\n---------------------------\n")


# 5. Define percentiles for binning
percentiles = [0, 0.0001, 0.0005, 0.01, 0.05, 0.2, 0.5, 0.8, 0.95, 0.99, 0.9999, 0.99995, 1.0]

# 6. Generate human-readable bin labels
bin_labels = [
    f"{percentiles[i]*100:.4f}–{percentiles[i+1]*100:.4f}%"
    for i in range(len(percentiles) - 1)
]

# 7. Create the binned variable on df1
df2['ratio_bin'] = pd.qcut(
    df2['ratio_to_vendor_mcc_median'],
    q=percentiles,
    labels=bin_labels
)

# 8. Count number of records in each bin
bin_counts = (
    df2['ratio_bin']
    .value_counts()
    .sort_index()
    .reset_index(name='count')
    .rename(columns={'index': 'ratio_bin'})
)

# 9. Plot with Plotly
fig = px.bar(
    bin_counts,
    x='ratio_bin',
    y='count',
    title='Transaction Count by Ratio-to-Median Percentile Bins',
    labels={'ratio_bin': 'Ratio Percentile Bin', 'count': 'Record Count'},
    template='plotly_white'
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()

print("\n---------------------------\n")

# 10. Show bin-level statistics
print("Ratio bins and their statistics relative to vendor-MCC median:")
print(df2.groupby('ratio_bin')['ratio_to_vendor_mcc_median']
        .agg(['count', 'min', 'max'])
        .reset_index())

print("\n---------------------------\n")

# 11. Inspect extreme ratios
print("Extremely low ratios (< 0):")
display(df2[df2['ratio_to_vendor_mcc_median'] < 0]
        .sort_values('ratio_to_vendor_mcc_median')
        .head())

print("\n---------------------------\n")

print("Extremely high ratios (> 100):")
display(df2[df2['ratio_to_vendor_mcc_median'] > 100]
        .sort_values('ratio_to_vendor_mcc_median', ascending=False)
        .head())


                         Vendor  \
87411             www.wiley.com   
87410     www.wff2014korea.org/   
87409          www.testking.com   
87408      www.sublimevideo.net   
87407           www.molport.com   
87406      www.eurofedlipid.org   
87405            www.autodoc.ch   
87404  wmill.com ResponseOmatic   
87403        value plastics inc   
87402       templatemonster.com   

                            Merchant Category Code (MCC)  vendor_mcc_median  
87411         BUSINESS SERVICES NOT ELSEWHERE CLASSIFIED           1,087.81  
87410  DIRCT MARKETING/DIRCT MARKETERS--NOT ELSEWHERE...             250.00  
87409         BUSINESS SERVICES NOT ELSEWHERE CLASSIFIED              99.99  
87408                           COMPUTER SOFTWARE STORES              19.90  
87407  CHEMICALS AND ALLIED PRODUCTS NOT ELSEWHERE CL...             205.00  
87406                  MISCELLANEOUS GENERAL MERCHANDISE           1,286.43  
87405                           COMPUTER SOFTWARE STORES            


---------------------------

Ratio bins and their statistics relative to vendor-MCC median:
            ratio_bin   count     min       max
0      0.0000–0.0100%      45 -663.00    -69.82
1      0.0100–0.0500%     177  -68.14    -15.83
2      0.0500–1.0000%    4203  -15.80     -0.94
3      1.0000–5.0000%   17698   -0.94      0.08
4     5.0000–20.0000%   66369    0.08      0.46
5    20.0000–50.0000%  183074    0.46      1.00
6    50.0000–80.0000%   82400    1.00      2.01
7    80.0000–95.0000%   66370    2.01      6.65
8    95.0000–99.0000%   17697    6.65     19.47
9    99.0000–99.9900%    4380   19.47    311.71
10   99.9900–99.9950%      22  313.79    459.73
11  99.9950–100.0000%      23  462.83 13,119.00

---------------------------

Extremely low ratios (< 0):


/var/folders/sb/pvs4j23j1z74y7s6nw77zrwr0000gn/T/ipykernel_3936/2683066731.py:80: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,Year-Month,Agency Number,Agency Name,Cardholder Last Name,Cardholder First Initial,Description,Amount,Vendor,Transaction Date,Posted Date,Merchant Category Code (MCC),vendor_mcc_median,ratio_to_vendor_mcc_median,ratio_bin
422223,2014-06-01,76000,UNIVERSITY OF OKLAHOMA,MCCASKILL,J,GENERAL PURCHASE,-6.63,GODADDY.COM CANADA,2014-06-19,2014-06-20,COMPUTER NETWORK/INFORMATION SERVICES,0.01,-663.00,0.0000–0.0100%
385422,2014-06-01,1000,OKLAHOMA STATE UNIVERSITY,Reece,S,GENERAL PURCHASE,"-1,876.34",BLUEBYTE TRADING LLC,2014-06-01,2014-06-03,MISCELLANEOUS AND SPECIALTY RETAIL STORES,2.84,-660.68,0.0000–0.0100%
251789,2013-10-01,98000,GRAND RIVER DAM AUTH.,Cole,K,GENERAL PURCHASE,"3,079.55",SEARS.COM 9300,2013-10-28,2013-10-30,DIRCT MARKETING/DIRCT MARKETERS--NOT ELSEWHERE...,-4.69,-657.32,0.0000–0.0100%
252126,2013-11-01,98000,GRAND RIVER DAM AUTH.,Stroup,P,GENERAL PURCHASE,"1,684.35",SEARS.COM 9300,2013-11-26,2013-11-29,DIRCT MARKETING/DIRCT MARKETERS--NOT ELSEWHERE...,-4.69,-359.52,0.0000–0.0100%
253984,2014-01-01,98000,GRAND RIVER DAM AUTH.,Stroup,P,GENERAL PURCHASE,"1,368.44",SEARS.COM 9300,2014-01-24,2014-01-27,DIRCT MARKETING/DIRCT MARKETERS--NOT ELSEWHERE...,-4.69,-292.09,0.0000–0.0100%



---------------------------

Extremely high ratios (> 100):


,Year-Month,Agency Number,Agency Name,Cardholder Last Name,Cardholder First Initial,Description,Amount,Vendor,Transaction Date,Posted Date,Merchant Category Code (MCC),vendor_mcc_median,ratio_to_vendor_mcc_median,ratio_bin
369304,2014-05-01,1000,OKLAHOMA STATE UNIVERSITY,Antene,M,ROOM CHARGES,131.19,COURTYARD BY MARRIOTT ADD,2014-05-27,2014-05-29,COURTYARD BY MARRIOTT,0.01,"13,119.00",99.9950–100.0000%
68603,2014-01-01,2500,OKLAHOMA MILITARY DEPARTMENT,MALLOY,J,GENERAL PURCHASE,"183,367.93",CONSTRUCTION DIVISION,2014-01-28,2014-01-29,GOVERNMENT SERVICES--NOT ELSEWHERE CLASSIFIED,127.49,"1,438.29",99.9950–100.0000%
251680,2013-10-01,98000,GRAND RIVER DAM AUTH.,Cardholder,e,GENERAL PURCHASE,"90,340.80",FAIRPOINT COMM. 80,2013-10-10,2013-10-11,GOVERNMENT SERVICES--NOT ELSEWHERE CLASSIFIED,98.35,918.52,99.9950–100.0000%
169572,2013-08-01,76000,UNIVERSITY OF OKLAHOMA,CANTRELL,P,ROOM CHARGES,"114,203.17",EMBASSY SUITES NORMAN,2013-08-08,2013-08-09,EMBASSY SUITES,165.00,692.14,99.9950–100.0000%
249956,2013-08-01,98000,GRAND RIVER DAM AUTH.,Hunter,G,CHARGE_TRANS-9182087817 SRV|CHARGE_TRANS-91820...,"63,748.30",SPRINT WIRELESS,2013-08-10,2013-08-12,TELECOMMUNICATION SERVICES,94.99,671.11,99.9950–100.0000%


This result looks much better than with using Mean. The results makes more sense and we are still able to identify some anomalies in the low and high extremes of transaction values against the Median vendor x MCC amount.

* There are 45 transactions with a ratio lower than -45. Ratio -45 is already concerning, but we found some with less than -100!
* There are 45 transactions with ratio higher than 313! This is also concerning and warrants further investigation.

### 3. Transaction Date vs Posted Date 

Let's look at the difference between the transaction date and the posted date. This can help us identify any delays in processing transactions (perhaps due to holidays, timezone differences, or other system errors which we must prevent). Delays in processing can be deterimental to merchants and even customers if they are expecting a transaction to be processed immediately, particularly for urgent matters.

In [18]:
df3 = df.copy()
# Compute the lag in days between transaction and posting
df3['lag_days'] = (df3['Posted Date'] - df3['Transaction Date']).dt.days

# Group by lag_days and compute min, mean, max of Amount
lag_stats = (
    df3.groupby('lag_days')['Amount']
      .agg(transaction_count='count', min_amount='min', mean_amount='mean', max_amount='max')
      .reset_index()
)

# Round mean_amount for readability
lag_stats['mean_amount'] = lag_stats['mean_amount'].round(2)

# Display the results
print(lag_stats)

    lag_days  transaction_count  min_amount  mean_amount   max_amount
0          0              18616   -3,889.57       564.50   373,150.26
1          1             201903  -41,740.00       392.57   348,053.75
2          2             106620  -38,506.87       416.23   403,490.75
3          3              76380  -34,108.00       437.41 1,089,180.00
4          4              31073  -21,000.00       467.03   814,934.76
..       ...                ...         ...          ...          ...
69       104                  2     -550.00      -425.97      -301.94
70       106                  1     -183.25      -183.25      -183.25
71       130                  1      -30.00       -30.00       -30.00
72       160                  1      -30.00       -30.00       -30.00
73       183                  1      -34.82       -34.82       -34.82

[74 rows x 5 columns]


Interestingly, even though these transactions are refunds, we have transactions with lags for more than 100 days! This can be cause for operational concern, as it indicates that there may be delays in processing refunds or that there are issues with the transaction data itself.

Below are the data with lags more than 100 days.

In [19]:
df3[df3['lag_days'] > 100].head(10)

,Year-Month,Agency Number,Agency Name,Cardholder Last Name,Cardholder First Initial,Description,Amount,Vendor,Transaction Date,Posted Date,Merchant Category Code (MCC),lag_days
31364,2013-10-01,1000,OKLAHOMA STATE UNIVERSITY,Tucker,E,GENERAL PURCHASE,-34.82,UNIVERSITEIT GENT,2013-04-17,2013-10-17,COMBINATION CATALOG AND RETAIL MERCHANT,183
32484,2013-10-01,1000,OKLAHOMA STATE UNIVERSITY,Beem,M,GENERAL PURCHASE,-301.94,Claim ADJ/STAPLES 0,2013-07-09,2013-10-21,"STATIONERY,OFFICE AND SCHOOL SUPPLY STORES",104
47627,2013-11-01,1000,OKLAHOMA STATE UNIVERSITY,Kaundal,R,GENERAL PURCHASE,-550.00,GLOBAL ENGAGE LTD,2013-07-31,2013-11-12,SCHOOLS AND EDUCATIONAL SERVICES NOT ELSEWHERE...,104
295987,2014-02-01,16000,DEPARTMENT OF COMMERCE,Lofton,L,ROOM CHARGES,-4.15,HOLIDAY INN EXPRESS,2013-11-12,2014-02-21,HOLIDAY INNS,101
329682,2014-02-01,76000,UNIVERSITY OF OKLAHOMA,MILLER,D,GENERAL PURCHASE,-30.00,AT&T DATA,2013-09-19,2014-02-26,TELECOMMUNICATION SERVICES,160
329683,2014-02-01,76000,UNIVERSITY OF OKLAHOMA,MILLER,D,GENERAL PURCHASE,-30.00,AT&T DATA,2013-10-19,2014-02-26,TELECOMMUNICATION SERVICES,130
405961,2014-06-01,53100,ROSE STATE COLEGE,Lastinger,S,GENERAL PURCHASE,-183.25,Claim ADJ/HOLIDAY INN DOW,2014-02-23,2014-06-09,"LODGING--HOTELS,MOTELS,AND RESORTS",106


### 4. Month on Month Change in Total Transaction Amount by Cardholder

This feature will help us identify cardholders who have a significant change in their total transaction amount from one month to the next. This can help us identify potential anomalies or unusual spending patterns. Another use case is, for future analysis, is to determine patterns for any sudden spikes in spending, e.g. during holidays or special events.

In [20]:
# 1. Compute monthly total spend per cardholder
df_cardholder_monthly = (
    df.groupby(['Cardholder Last Name', 'Cardholder First Initial', 'Year-Month'])['Amount']
      .sum()
      .reset_index(name='monthly_spend')
      .sort_values(['Cardholder Last Name', 'Cardholder First Initial', 'Year-Month'])
)

# Round the total amount
df_cardholder_monthly['monthly_spend'] = df_cardholder_monthly['monthly_spend'].round(2)

# Filter out months with 0 spend
df_cardholder_monthly = df_cardholder_monthly[df_cardholder_monthly['monthly_spend'] != 0]

df_cardholder_monthly = df_cardholder_monthly.sort_values(['Cardholder Last Name', 'Cardholder First Initial', 'Year-Month'])

# 3. Compute month-on-month percent change per cardholder
df_cardholder_monthly['pct_change'] = (
    df_cardholder_monthly
    .groupby(['Cardholder Last Name', 'Cardholder First Initial'])['monthly_spend']
    .pct_change() * 100
)

# 4. Extract and clean percent changes
pct_changes = df_cardholder_monthly['pct_change'].dropna()

df_cardholder_monthly['pct_change'].describe()

count       36,876.00
mean           251.62
std          7,724.39
min     -1,352,100.00
25%            -58.27
50%             -1.49
75%            124.21
max        184,508.54
Name: pct_change, dtype: float64

We can see that there are some customers who has made significant changes in their spending patterns from one month to the next. This could indicate potential anomalies or unusual spending patterns.

In [21]:
# Get two users with the largest negative percent change
lowest_changes = df_cardholder_monthly[df_cardholder_monthly['pct_change'] < -100].sort_values('pct_change').head(2)[['Cardholder Last Name', 'Cardholder First Initial']]
display(df_cardholder_monthly.merge(lowest_changes, on=['Cardholder Last Name', 'Cardholder First Initial'], how='inner'))

,Cardholder Last Name,Cardholder First Initial,Year-Month,monthly_spend,pct_change
0,FEINBERG,R,2013-07-01,409.50,NaN
1,FEINBERG,R,2013-08-01,482.55,17.84
2,FEINBERG,R,2013-09-01,260.78,-45.96
3,FEINBERG,R,2013-10-01,332.37,27.45
4,FEINBERG,R,2013-11-01,-0.16,-100.05
5,FEINBERG,R,2014-01-01,509.32,"-318,425.00"
6,FEINBERG,R,2014-02-01,736.86,44.68
7,FEINBERG,R,2014-03-01,205.77,-72.07
8,FEINBERG,R,2014-05-01,6.25,-96.96
9,FEINBERG,R,2014-06-01,64.56,932.96


In [22]:
# Get two users with the largest percent change
highest_changes = df_cardholder_monthly[df_cardholder_monthly['pct_change'] > 1000].sort_values('pct_change', ascending=False).head(2)[['Cardholder Last Name', 'Cardholder First Initial']]
display(df_cardholder_monthly.merge(highest_changes, on=['Cardholder Last Name', 'Cardholder First Initial'], how='inner'))

,Cardholder Last Name,Cardholder First Initial,Year-Month,monthly_spend,pct_change
0,Antene,M,2013-07-01,"1,042.46",NaN
1,Antene,M,2013-08-01,"1,047.90",0.52
2,Antene,M,2013-09-01,2.80,-99.73
3,Antene,M,2013-10-01,"5,025.17","179,370.36"
4,Antene,M,2013-11-01,"1,910.59",-61.98
5,Antene,M,2013-12-01,"2,599.70",36.07
6,Antene,M,2014-01-01,"2,096.28",-19.36
7,Antene,M,2014-02-01,"2,649.06",26.37
8,Antene,M,2014-03-01,782.47,-70.46
9,Antene,M,2014-04-01,"4,772.31",509.90


We can see that these transactions are quite sudden and significant, with some customers spending more than 1000% more in one month compared to the previous month. This could indicate potential anomalies or unusual spending patterns.

In [23]:
unique_cardholders = df[['Cardholder Last Name', 'Cardholder First Initial']].drop_duplicates()
highest_changes_cardholders = df_cardholder_monthly[df_cardholder_monthly['pct_change'] > 1000][['Cardholder Last Name', 'Cardholder First Initial']].drop_duplicates()

print(f"Total unique cardholders: {unique_cardholders.shape[0]:,}")
print(f"Total unique cardholders with higher Month on Month Transaction Value: {highest_changes_cardholders.shape[0]:,} ({highest_changes_cardholders.shape[0] / unique_cardholders.shape[0] * 100:.2f}%)")

Total unique cardholders: 5,213
Total unique cardholders with higher Month on Month Transaction Value: 1,723 (33.05%)


According to this rule, about 33.05% of cardholders have made significant changes. This maybe because the threshold is still considered normal in this particular user base and industry. We can try to increase the threshold to see if we can find more anomalies.

In [24]:
unique_cardholders = df[['Cardholder Last Name', 'Cardholder First Initial']].drop_duplicates()
highest_changes_cardholders = df_cardholder_monthly[df_cardholder_monthly['pct_change'] > 5000][['Cardholder Last Name', 'Cardholder First Initial']].drop_duplicates()

print(f"Total unique cardholders: {unique_cardholders.shape[0]:,}")
print(f"Total unique cardholders with higher Month on Month Transaction Value: {highest_changes_cardholders.shape[0]:,} ({highest_changes_cardholders.shape[0] / unique_cardholders.shape[0] * 100:.2f}%)")

Total unique cardholders: 5,213
Total unique cardholders with higher Month on Month Transaction Value: 398 (7.63%)


Increasing the threshold to 5000% resulted in 7.6% of cardholders considered as anomalies. This increased the focus on transaction value change. Again, the one to decide what threshold to use is up to the business and the context of the data.

This feature can be used to identify users and times in the dataset when they might make higher transactions than usual, combined with other information such as the vendor, MCC, and transaction date. This can help us identify and predict potential anomalies or unusual spending patterns.

### 5. Month on Month Transaction Amount Change by Vendor

Similar to the previous analysis, we can see that there are some vendors who have made significant changes in their charge patterns from one month to the next. This could indicate potential anomalies or unusual charging patterns.

In [25]:

# 1. Compute monthly total spend per vendor
df_vendor_monthly = (
    df.groupby(['Vendor', 'Year-Month'])['Amount']
      .sum()
      .reset_index(name='monthly_spend')
      .sort_values(['Vendor', 'Year-Month'])
)

# 2. Round the total amount
df_vendor_monthly['monthly_spend'] = df_vendor_monthly['monthly_spend'].round(2)

# 3. Filter out months with zero spend
df_vendor_monthly = df_vendor_monthly[df_vendor_monthly['monthly_spend'] != 0]

# 4. Compute month-on-month percent change per vendor
df_vendor_monthly['pct_change'] = (
    df_vendor_monthly
    .groupby('Vendor')['monthly_spend']
    .pct_change() * 100
)

# 5. Show descriptive statistics for the percent changes
print(df_vendor_monthly['pct_change'].describe())


count       59,639.00
mean           268.72
std         76,569.01
min     -6,666,766.67
25%            -56.52
50%              0.00
75%            100.00
max     17,137,500.00
Name: pct_change, dtype: float64


In [26]:
display(df_vendor_monthly[df_vendor_monthly['Vendor'] == 'IRISHHOUSEPARTY COM'])
display(df_vendor_monthly.merge(df_vendor_monthly[df_vendor_monthly['pct_change'] < -1000000][['Vendor']].drop_duplicates(), on='Vendor', how='inner'))

,Vendor,Year-Month,monthly_spend,pct_change
71076,IRISHHOUSEPARTY COM,2014-04-01,0.01,NaN
71077,IRISHHOUSEPARTY COM,2014-05-01,"1,713.76","17,137,500.00"


,Vendor,Year-Month,monthly_spend,pct_change
0,SHERATON SEATTLE HOTEL,2013-09-01,"1,511.20",NaN
1,SHERATON SEATTLE HOTEL,2013-10-01,"5,067.70",235.34
2,SHERATON SEATTLE HOTEL,2013-11-01,-0.02,-100.00
3,SHERATON SEATTLE HOTEL,2013-12-01,377.80,"-1,889,100.00"
4,SHERATON SEATTLE HOTEL,2014-02-01,408.60,8.15
5,SHERATON SEATTLE HOTEL,2014-04-01,710.84,73.97
6,SHERATON SEATTLE HOTEL,2014-06-01,"2,652.41",273.14
7,UNIV OF OK BKST #831,2013-08-01,455.64,NaN
8,UNIV OF OK BKST #831,2013-09-01,57.50,-87.38
9,UNIV OF OK BKST #831,2013-10-01,-31.64,-155.03


For Vendors we can see that the extreme cases are done by some vendors who just happen to have some steady transaction values and likely due to seaswonality factors, they had a jump in revenue.

Let's exclude these vendors and see if we can find actual anomalies.


In [27]:
extreme_vendors = (
    df_vendor_monthly[
        (df_vendor_monthly['pct_change'] < -1_000_000) |
        (df_vendor_monthly['pct_change'] >  1_000_000)
    ]['Vendor']
    .unique()
)

df_vendor_monthly_2 = df_vendor_monthly[~df_vendor_monthly['Vendor'].isin(extreme_vendors)]

print(df_vendor_monthly_2['pct_change'].describe())


count     59,621.00
mean         170.84
std        2,665.80
min     -247,490.64
25%          -56.50
50%            0.00
75%          100.00
max      208,217.19
Name: pct_change, dtype: float64


Now the numbers look much better. Let us see who these vendors are anomalous according to this method.

In [28]:
unique_vendors = df[['Vendor']].drop_duplicates()
highest_changes_vendors = df_vendor_monthly_2[(df_vendor_monthly_2['pct_change'] > 200000) | (df_vendor_monthly_2['pct_change'] < -200000 )][['Vendor']].drop_duplicates()

print(f"Total unique vendors: {unique_vendors.shape[0]:,}")
print(f"Total unique vendors with higher Month on Month Transaction Value CHange: {highest_changes_vendors.shape[0]:,} ({highest_changes_vendors.shape[0] / unique_vendors.shape[0] * 100:.2f}%)")
display(df_vendor_monthly_2[(df_vendor_monthly_2['pct_change'] > 200000) | (df_vendor_monthly_2['pct_change'] < -200000) ].head(10))

Total unique vendors: 86,729
Total unique vendors with higher Month on Month Transaction Value CHange: 3 (0.00%)


,Vendor,Year-Month,monthly_spend,pct_change
40657,DAVIS PIPE & SUPPLY-NORMA,2014-02-01,"1,333.23","208,217.19"
101770,OSWALT RESTUARANT SUPPLY,2013-08-01,"4,230.38","-247,490.64"
116774,SONOSITE INC,2014-01-01,"44,332.00","-201,334.68"


These are the vendors that have made significant changes in their charging patterns from one month to the next. This could indicate potential anomalies or unusual charging patterns.

### 6. Average Monthly Transaction Amount and Standard Deviation by Vendor

We can add another layer in our logic to make the previous analysis more robust. 
* We can add the average monthly transaction amount for each vendor as well as its standard deviation. This will allow us to identify vendors who have made significant changes in their charging patterns from one month to the next, while also taking into account the average monthly transaction amount and its variability.

* Another layer of filter we can add is to only consider vendors with a minimum of 3-5 monthly transactions. This will help us filter out vendors with very few transactions, which may not be representative of their overall charging patterns.

* Another layer of filter is to oncly consider transactions amounts higher than let's say 5000. This will help us filter out small transactions that may not be representative of the overall charging patterns.

In [29]:
df_vendor_monthly = (
    df
    .groupby(['Vendor', 'Year-Month'])['Amount']
    .agg(monthly_tot = 'sum')
    .reset_index()
)

vendor_dist = df.groupby('Vendor')['Amount'].agg(monthly_avg='mean', monthly_std='std').reset_index()
vendor_nmonth = df_vendor_monthly.groupby('Vendor')['Year-Month'].nunique().reset_index(name='n_months')
df_vendor_monthly = df_vendor_monthly.merge(vendor_nmonth[vendor_nmonth['n_months'] > 3], on='Vendor', how='inner').merge(
    vendor_dist, on='Vendor', how='left')

df_vendor_monthly['anomaly'] = (
    (df_vendor_monthly['monthly_tot'] - df_vendor_monthly['monthly_avg']).abs() > 7 * df_vendor_monthly['monthly_std']
) & (df_vendor_monthly['monthly_tot'] > 5000)

display(df_vendor_monthly[df_vendor_monthly['anomaly']].head(10))

,Vendor,Year-Month,monthly_tot,n_months,monthly_avg,monthly_std,anomaly
243,4IMPRINT,2013-07-01,"10,566.67",12,756.21,863.32,True
244,4IMPRINT,2013-08-01,"9,301.63",12,756.21,863.32,True
246,4IMPRINT,2013-10-01,"16,261.84",12,756.21,863.32,True
248,4IMPRINT,2013-12-01,"7,519.07",12,756.21,863.32,True
251,4IMPRINT,2014-03-01,"7,729.15",12,756.21,863.32,True
252,4IMPRINT,2014-04-01,"10,482.08",12,756.21,863.32,True
253,4IMPRINT,2014-05-01,"9,738.36",12,756.21,863.32,True
254,4IMPRINT,2014-06-01,"13,437.73",12,756.21,863.32,True
332,820 NEWS PRESS,2013-08-01,"5,508.42",12,227.98,233.05,True
340,820 NEWS PRESS,2014-04-01,"6,972.76",12,227.98,233.05,True


In [30]:
unique_vendors = df[['Vendor']].drop_duplicates()
highest_changes_vendors = df_vendor_monthly[df_vendor_monthly['anomaly']][['Vendor']].drop_duplicates()

print(f"Total unique vendors: {unique_vendors.shape[0]:,}")
print(f"Total unique vendors with higher Monthly Transaction Value: {highest_changes_vendors.shape[0]:,} ({highest_changes_vendors.shape[0] / unique_vendors.shape[0] * 100:.2f}%)")

Total unique vendors: 86,729
Total unique vendors with higher Monthly Transaction Value: 693 (0.80%)


The analysis shows that only a very small fraction of vendors—0.80%—exhibit unusually high monthly transaction values according to our dual criteria (beyond 7 σ of their own historical spend and above \$5 000 in total).

* Vendor transaction volumes are generally stable, so spikes stand out clearly
* The 693 flagged vendors can possibly include new or one-off large contracts, bulk orders, or billing errors
* Retail giants or recurring suppliers rarely breach these thresholds, reinforcing that these anomalies reflect exceptional activity
* Focusing investigation on this 0.8 % can efficiently surface high-risk vendors without overwhelming review teams
* Comparing this to cardholder anomalies (27 % of users) underscores that vendor-level monitoring is more targeted and resource-efficient


### 7. Refund vs. Purchase Patterns

We can check to see in one year, customers who have made similar or even higher amount of refunds compared to their purchases. This can help us identify any potential anomalies or unusual spending patterns.

In [31]:

# 1. Make a copy of the original DataFrame
df5 = df.copy()

# 2. Ensure Amount is numeric
df5['Amount'] = pd.to_numeric(df5['Amount'], errors='coerce')

# 3. Identify purchases (positive) vs refunds (negative)
df5['is_refund']   = df5['Amount'] < 0
df5['is_purchase'] = df5['Amount'] > 0

# 4. Aggregate per cardholder–month
refunds_vs_purchases = (
    df5
    .groupby(['Cardholder Last Name', 'Cardholder First Initial', 'Year-Month'])
    .agg(
        total_purchases      = ('is_purchase', 'sum'),
        total_refunds        = ('is_refund', 'sum'),
        amt_purchased        = ('Amount', lambda x: x[x>0].sum()),
        amt_refunded         = ('Amount', lambda x: x[x<0].sum()),
        transaction_count    = ('Amount', 'count'),
    )
    .reset_index()
)

# 5. Compute ratios
refunds_vs_purchases['refund_txn_ratio'] = (
    refunds_vs_purchases['total_refunds'] / refunds_vs_purchases['transaction_count']
)
refunds_vs_purchases['refund_amt_ratio'] = (
    refunds_vs_purchases['amt_refunded'].abs() 
    / (refunds_vs_purchases['amt_purchased'] + refunds_vs_purchases['amt_refunded'].abs())
)

# 6. Flag anomalous months - the thresholds here are at the moment arbitrary and can be adjusted
refunds_vs_purchases['high_refund_rate'] = refunds_vs_purchases['refund_txn_ratio'] > 0.5
refunds_vs_purchases['large_refund_amt'] = refunds_vs_purchases['refund_amt_ratio'] > 0.5

potential_anomalies = refunds_vs_purchases[(refunds_vs_purchases['high_refund_rate'] == True) & 
                           (refunds_vs_purchases['large_refund_amt'] == True) &
                           (refunds_vs_purchases['amt_refunded'] < -300)
                           ].sort_values('amt_refunded', ascending=False)

# 7. Inspect the result
print(f"Potential anomalies found: {potential_anomalies.shape[0]:,} rows")
display(potential_anomalies.head(20))


Potential anomalies found: 73 rows


,Cardholder Last Name,Cardholder First Initial,Year-Month,total_purchases,total_refunds,amt_purchased,amt_refunded,transaction_count,refund_txn_ratio,refund_amt_ratio,high_refund_rate,large_refund_amt
33760,SUBRAMANIAN,A,2014-05-01,0,1,0.00,-303.50,1,1.00,1.00,True,True
7139,Castle,K,2013-11-01,0,1,0.00,-325.00,1,1.00,1.00,True,True
26653,Moreau,M,2014-05-01,0,1,0.00,-331.40,1,1.00,1.00,True,True
13585,Gambrell,S,2013-07-01,0,1,0.00,-337.74,1,1.00,1.00,True,True
14033,Glasgow,S,2014-02-01,0,1,0.00,-348.50,1,1.00,1.00,True,True
26521,Moody,D,2013-11-01,0,1,0.00,-350.00,1,1.00,1.00,True,True
2522,BRYANT,J,2014-02-01,0,1,0.00,-359.72,1,1.00,1.00,True,True
28448,PAGE,M,2014-03-01,0,1,0.00,-361.00,1,1.00,1.00,True,True
7057,Cartmell,D,2013-08-01,0,1,0.00,-365.88,1,1.00,1.00,True,True
24976,Mason,C,2013-10-01,0,1,0.00,-417.00,1,1.00,1.00,True,True


We can see that there are some customers with very high amounts and rates of refunds vs purcahases. The thresholds used here are currently arbitrary and would need to be confimred with the business. But this methodology can be used on a customer/user level to flag potential anomalies in how certain users behave in the data.

### 8. Refunds by Vendor
We can also check to see which vendors have the highest amount of refunds. This can help us identify any potential anomalies or unusual spending patterns.

In [32]:
# 1. Make a copy of the original DataFrame
df6 = df.copy()

# 2. Ensure Amount is numeric
df6['Amount'] = pd.to_numeric(df6['Amount'], errors='coerce')

# 3. Identify purchases (positive) vs refunds (negative)
df6['is_refund']   = df6['Amount'] < 0
df6['is_purchase'] = df6['Amount'] > 0

# 4. Aggregate per Vendor–month
refunds_vs_purchases_vendor = (
    df6
    .groupby(['Vendor', 'Year-Month'])
    .agg(
        total_purchases      = ('is_purchase', 'sum'),
        total_refunds        = ('is_refund', 'sum'),
        amt_purchased        = ('Amount', lambda x: x[x>0].sum()),
        amt_refunded         = ('Amount', lambda x: x[x<0].sum()),
        transaction_count    = ('Amount', 'count'),
    )
    .reset_index()
)

# 5. Compute ratios
refunds_vs_purchases_vendor['refund_txn_ratio'] = (
    refunds_vs_purchases_vendor['total_refunds'] 
    / refunds_vs_purchases_vendor['transaction_count']
)
refunds_vs_purchases_vendor['refund_amt_ratio'] = (
    refunds_vs_purchases_vendor['amt_refunded'].abs() 
    / (refunds_vs_purchases_vendor['amt_purchased'] 
       + refunds_vs_purchases_vendor['amt_refunded'].abs())
)

# 6. Flag anomalous months (adjust thresholds as needed)
refunds_vs_purchases_vendor['high_refund_rate'] = (
    refunds_vs_purchases_vendor['refund_txn_ratio'] > 0.5
)
refunds_vs_purchases_vendor['large_refund_amt'] = (
    refunds_vs_purchases_vendor['refund_amt_ratio'] > 0.5
)

# Identify potential anomalies
potential_anomalies_vendor = refunds_vs_purchases_vendor[
    (refunds_vs_purchases_vendor['high_refund_rate']) &
    (refunds_vs_purchases_vendor['large_refund_amt']) &
    (refunds_vs_purchases_vendor['amt_refunded'] < -300)
].sort_values('amt_refunded', ascending=False)

# 7. Inspect the result
print(f"Potential vendor-month anomalies found: {potential_anomalies_vendor.shape[0]:,} rows")
print(potential_anomalies_vendor.head(20))


Potential vendor-month anomalies found: 945 rows
                           Vendor Year-Month  total_purchases  total_refunds  \
40105   Claim ADJ/STAPLES       0 2013-10-01                0              1   
102849       PARTY PICS USA, INC. 2013-07-01                0              3   
115501                SHOEBUY.COM 2014-04-01                3              6   
29734          BILTMORE HOTEL OKC 2014-01-01                0              1   
11875   AMERICAN AI 0017284459521 2013-07-01                0              1   
15736   AMERICAN AI 0017316324478 2013-12-01                0              1   
16200   AMERICAN AI 0017363897459 2013-12-01                0              1   
15734   AMERICAN AI 0017316324477 2013-12-01                0              1   
15732   AMERICAN AI 0017316324476 2013-12-01                0              1   
65523     HGSE PROGRAM IN PROF ED 2014-04-01                0              1   
145400             WWW WURTEC COM 2013-08-01                0          

We can see that there are some vendors with very high amounts and rates of refunds vs purcahases. The thresholds used here are currently arbitrary and would need to be confimred with the business. But this methodology can be used to on vendor level to flag potential anomalies in how certain vendors behave in the data.

### 9. Transaction Velocity

Count per Time Window: Number of transactions per cardholder per day. Sudden spikes (e.g., 20 purchases in one day) may indicate fraud or system errors.

In [33]:
# 1. Make a copy of the DataFrame
df_velocity = df.copy()

# 2. Extract date and week identifiers
df_velocity['txn_date'] = df_velocity['Transaction Date'].dt.date
df_velocity['txn_week'] = df_velocity['Transaction Date'].dt.to_period('W').apply(lambda r: r.start_time.date())

# 3. Compute daily transaction velocity per cardholder
daily_velocity = (
    df_velocity
    .groupby(['Cardholder Last Name', 'Cardholder First Initial', 'txn_date'])
    .size()
    .reset_index(name='txns_per_day')
)

# 4. Compute weekly transaction velocity per cardholder
weekly_velocity = (
    df_velocity
    .groupby(['Cardholder Last Name', 'Cardholder First Initial', 'txn_week'])
    .size()
    .reset_index(name='txns_per_week')
)

# 5. Flag high-velocity days/weeks (e.g., > mean + 4*std, this number is arbitrary and can be adjusted)
stats_daily = daily_velocity.groupby(['Cardholder Last Name', 'Cardholder First Initial'])['txns_per_day'].agg(['mean','std']).reset_index()
daily_velocity = daily_velocity.merge(stats_daily, on=['Cardholder Last Name','Cardholder First Initial'])
daily_velocity['day_velocity_anomaly'] = daily_velocity['txns_per_day'] > (daily_velocity['mean'] + 4*daily_velocity['std'])

stats_weekly = weekly_velocity.groupby(['Cardholder Last Name', 'Cardholder First Initial'])['txns_per_week'].agg(['mean','std']).reset_index()
weekly_velocity = weekly_velocity.merge(stats_weekly, on=['Cardholder Last Name','Cardholder First Initial'])
weekly_velocity['week_velocity_anomaly'] = weekly_velocity['txns_per_week'] > (weekly_velocity['mean'] + 4*weekly_velocity['std'])

# 6. Inspect anomalies
print(f"Daily velocity anomalies: {daily_velocity[daily_velocity['day_velocity_anomaly']].shape[0]:,} rows")
print(daily_velocity[daily_velocity['day_velocity_anomaly']].head())

print(f"\nWeekly velocity anomalies:{weekly_velocity[weekly_velocity['week_velocity_anomaly']].shape[0]:,} rows")
print(weekly_velocity[weekly_velocity['week_velocity_anomaly']].head())


Daily velocity anomalies: 1,702 rows
    Cardholder Last Name Cardholder First Initial    txn_date  txns_per_day  \
128                ADAMS                        K  2013-07-11            12   
201                  ADE                        D  2013-09-14             3   
428               ALFORD                        R  2013-10-10             6   
443               ALFORD                        R  2014-01-21             6   
638     ALL ABOUT TRAVEL                        G  2014-02-14            22   

     mean  std  day_velocity_anomaly  
128  1.45 1.72                  True  
201  1.13 0.39                  True  
428  1.31 1.12                  True  
443  1.31 1.12                  True  
638  3.91 4.45                  True  

Weekly velocity anomalies:234 rows
     Cardholder Last Name Cardholder First Initial    txn_week  txns_per_week  \
107                 ADAMS                        K  2013-07-08             14   
620              ANDERSON                        B  2014

We have found multiple users across multiple months with higher than usual transaction counts per day and per week. This could indicate potential anomalies or unusual spending patterns. We can go deeper and see which users have the highest transaction counts per day and per week.

In [34]:
unique_cardholders = df[['Cardholder Last Name', 'Cardholder First Initial']].drop_duplicates()
unique_cardholders_day = daily_velocity[daily_velocity['day_velocity_anomaly']][['Cardholder Last Name', 'Cardholder First Initial']].drop_duplicates()
unique_cardholders_week = weekly_velocity[weekly_velocity['week_velocity_anomaly']][['Cardholder Last Name', 'Cardholder First Initial']].drop_duplicates()

print(f"Total unique cardholders: {unique_cardholders.shape[0]:,}")
print(f"Total unique cardholders with higher than usual daily trainsaction velocity: {unique_cardholders_day.shape[0]:,} ({unique_cardholders_day.shape[0] / unique_cardholders.shape[0] * 100:.2f}%)")
print(f"Total unique cardholders with higher than usual weekly trainsaction velocity: {unique_cardholders_week.shape[0]:,} ({unique_cardholders_week.shape[0] / unique_cardholders.shape[0] * 100:.2f}%)")

Total unique cardholders: 5,213
Total unique cardholders with higher than usual daily trainsaction velocity: 1,419 (27.22%)
Total unique cardholders with higher than usual weekly trainsaction velocity: 232 (4.45%)


We can see that there are some users with very high transaction counts per day and per week. 

Moving from weekly to daily dropped the number of anomalous users significantly from 27.2% to 4.45% making it more focused. 

The thresholds used here are currently arbitrary and would need to be confimred with the business. But this methodology can be used to flag potential anomalies in how certain users behave in the data.

### 10. Transaction Velocities by Vendor

Same as the previous analysis, but at the vendor level. This can help us identify any potential anomalies or unusual charging patterns.

In [35]:
import pandas as pd

# 1. Copy the DataFrame
df7 = df.copy()

# 2. Extract date and week
df7['txn_date'] = df7['Transaction Date'].dt.date
df7['txn_week'] = df7['Transaction Date'].dt.to_period('W').apply(lambda r: r.start_time.date())

# 3. Compute daily transaction velocity per vendor
daily_vendor_vel = (
    df7
    .groupby(['Vendor', 'txn_date'])
    .size()
    .reset_index(name='txns_per_day')
)

# 4. Compute weekly transaction velocity per vendor
weekly_vendor_vel = (
    df7
    .groupby(['Vendor', 'txn_week'])
    .size()
    .reset_index(name='txns_per_week')
)

# 5. Calculate each vendor’s historical mean & std
daily_stats = daily_vendor_vel.groupby('Vendor')['txns_per_day'].agg(['mean','std']).reset_index()
weekly_stats = weekly_vendor_vel.groupby('Vendor')['txns_per_week'].agg(['mean','std']).reset_index()

# 6. Merge stats back and flag anomalies
daily_vendor_vel = daily_vendor_vel.merge(daily_stats, on='Vendor')
daily_vendor_vel['day_velocity_anomaly'] = (
    daily_vendor_vel['txns_per_day'] > (daily_vendor_vel['mean'] + 4 * daily_vendor_vel['std'])
)

weekly_vendor_vel = weekly_vendor_vel.merge(weekly_stats, on='Vendor')
weekly_vendor_vel['week_velocity_anomaly'] = (
    weekly_vendor_vel['txns_per_week'] > (weekly_vendor_vel['mean'] + 4 * weekly_vendor_vel['std'])
)

# 8. Inspect anomalies
print(f"Daily velocity vendor anomalies: {daily_vendor_vel[daily_vendor_vel['day_velocity_anomaly']].shape[0]:,} rows")
print(daily_vendor_vel[daily_vendor_vel['day_velocity_anomaly']].head())

print(f"Weekly velocity vendor anomalies: {weekly_vendor_vel[weekly_vendor_vel['week_velocity_anomaly']].shape[0]:,} rows")
print(weekly_vendor_vel[weekly_vendor_vel['week_velocity_anomaly']].head())


Daily velocity vendor anomalies: 851 rows
                     Vendor    txn_date  txns_per_day  mean  std  \
228       159863 CARMIKE 10  2014-01-05             3  1.08 0.41   
484         3D ROBOTICS INC  2014-01-31             2  1.03 0.18   
903   500 NORMAN TRANSCRIPT  2014-05-06             4  1.19 0.61   
1177         820 NEWS PRESS  2014-05-08             7  1.51 0.89   
1600      A PLUS AUTO GLASS  2013-08-12             7  1.59 1.28   

      day_velocity_anomaly  
228                   True  
484                   True  
903                   True  
1177                  True  
1600                  True  
Weekly velocity vendor anomalies: 119 rows
                          Vendor    txn_week  txns_per_week  mean  std  \
2582   ACCURATE ENVIROMENTAL LAB  2013-10-14             20  4.96 3.18   
2712                ACE HARDWARE  2014-01-13              6  1.67 1.00   
11040  AIRCRAFT SPRUCE AND SPECI  2014-01-13              5  1.52 0.85   
11333     ALARMCENTER 8664844800  20

We have also found some vendors with very high transaction velocities. 

In [36]:
unique_vendors = df[['Vendor']].drop_duplicates()
unique_vendors_day = daily_vendor_vel[daily_vendor_vel['day_velocity_anomaly']][['Vendor']].drop_duplicates()
unique_vendors_week = weekly_vendor_vel[weekly_vendor_vel['week_velocity_anomaly']][['Vendor']].drop_duplicates()

print(f"Total unique Vendors: {unique_vendors.shape[0]:,}")
print(f"Total unique Vendors with higher than usual daily trainsaction velocity: {unique_vendors_day.shape[0]:,} ({unique_vendors_day.shape[0] / unique_vendors.shape[0] * 100:.2f}%)")
print(f"Total unique Vendors with higher than usual weekly trainsaction velocity: {unique_vendors_week.shape[0]:,} ({unique_vendors_week.shape[0] / unique_vendors.shape[0] * 100:.2f}%)")

Total unique Vendors: 86,729
Total unique Vendors with higher than usual daily trainsaction velocity: 754 (0.87%)
Total unique Vendors with higher than usual weekly trainsaction velocity: 112 (0.13%)


The data show that anomalous spikes in transaction velocity are far more prevalent among individual cardholders than among vendors. Only a tiny fraction of vendors break their own daily or weekly volume patterns, while over a quarter of cardholders exceed their personal norms on a daily basis.

* Vendor anomalies are rare: 0.87 % exceed daily thresholds and 0.13 % exceed weekly thresholds.
* Cardholder anomalies are common: 27.22 % exceed daily thresholds and 4.45 % exceed weekly thresholds.
* Large vendors have high, stable baselines—random fluctuations rarely stand out.
* Cardholders’ activity varies with travel, projects, and one-off events, so bursty behavior is expected.
* Prioritize velocity monitoring at the cardholder level to catch most unusual activity.
* Use vendor velocity alerts as a secondary check for sudden new or rogue suppliers.


### 11. Time since last transaction

In [41]:
import pandas as pd
import plotly.express as px

# 1. Copy & prepare data
df11 = df.copy()
df11 = df11.sort_values(
    ['Cardholder Last Name', 'Cardholder First Initial', 'Transaction Date']
).reset_index(drop=True)
df11['days_since_last_txn'] = df11.groupby(
    ['Cardholder Last Name', 'Cardholder First Initial']
)['Transaction Date'].diff().dt.days

# 2. Drop NaNs (first txn per cardholder)
data = df11['days_since_last_txn'].dropna()

# 3. Define your desired quantiles
quantiles = [0, 0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99, 0.999, 0.9999, 1.0]

# 4. Bin with qcut (let it auto-label, dropping duplicate edges)
binned = pd.qcut(data, q=quantiles, duplicates='drop')

# 5. Count per bin
bin_counts = (
    binned
    .value_counts()
    .sort_index()
    .reset_index(name='count')
    .rename(columns={'index': 'percentile_bin'})
)

# 6. Convert the Interval objects to strings for plotting
bin_counts['days_since_last_txn'] = bin_counts['days_since_last_txn'].astype(str)

# 7. Plot with Plotly
fig = px.bar(
    bin_counts,
    x='days_since_last_txn',
    y='count',
    title='Distribution of Days Since Last Transaction (Percentile Bins)',
    labels={'days_since_last_txn': 'Days-Since-Last-Txn Bin', 'count': 'Transaction Count'},
    template='plotly_white'
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


In [42]:
bin_counts

,days_since_last_txn,count
0,"(-0.001, 1.0]",295799
1,"(1.0, 3.0]",49906
2,"(3.0, 7.0]",48390
3,"(7.0, 14.0]",23059
4,"(14.0, 38.0]",15753
5,"(38.0, 129.0]",3908
6,"(129.0, 240.551]",386
7,"(240.551, 367.0]",44



Key observations

* Same-day repeats dominate usage (≈75%)
* 1–3 day and 3–7 day gaps each cover about 12% of transactions
* Monthly-cycle gaps (7–14 days, 14–38 days) account for under 10%
* Gaps over 38 days are under 1% and warrant anomaly review
* Outliers beyond 129 days are extremely rare and should be flagged for investigation, 
* Even more so are the users who take almost a year to make a new transaction. These are the prime candidates for anomalies (whether normal or an area of improvement for business) and for deeper investigation.

This feature is helpful to analyze and predict on a customer level. As an illustration, below are the users who have not made a transaction in more than 240 days.


In [ ]:
df11[df11['days_since_last_txn'] > 240].sort_values(by='days_since_last_txn', ascending=False).groupby(['Cardholder Last Name', 'Cardholder First Initial'])['days_since_last_txn'].max().reset_index()


,Cardholder Last Name,Cardholder First Initial,days_since_last_txn
0,77000,7,261.00
1,BIRKETT,L,277.00
2,BUGHER,J,352.00
3,Basco,T,367.00
4,Blewett,E,281.00
5,Blum,F,271.00
6,Bowser,T,259.00
7,Boyd,B,286.00
8,Brown,G,314.00
9,Chewey,D,320.00


## Conclusion

Our analysis demonstrates that combining fundamental spend summaries with anomaly-centric features offers a powerful lens for detecting irregularities in purchase-card programs. While most transactions fall within expected patterns—same-day repeats, short inter-transaction gaps, and stable vendor relationships—a small fraction exhibits behaviors characteristic of misuse or process breakdowns. Automated flagging based on velocity, refund proportions, and deviation from historical medians can prioritize investigative efforts. Moving forward, integrating these analytics into routine oversight will strengthen internal controls, enhance compliance, and protect the bank's customers.
